In [14]:
import os 
import pyttsx3
from ebooklib import epub
from bs4 import BeautifulSoup
import fitz
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import threading
import json


In [15]:
# Initialize pyttsx3 engine
engine = pyttsx3.init()

In [16]:
# Set default properties
engine.setProperty('rate', 150)    # Speech rate
engine.setProperty('volume', 1.0)  # Volume (0.0 to 1.0)

In [17]:
# Global variable to hold the current text
current_text = ""

In [18]:
voices = engine.getProperty('voices')

for index, voice in enumerate(voices):
    print(f"Voice {index}: {voice.name} - {voice.languages}")

Voice 0: Microsoft David Desktop - English (United States) - []
Voice 1: Microsoft Zira Desktop - English (United States) - []


In [19]:
## Function to read .txt files
def read_text_file(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as file:
        return file.read()

In [20]:
# Function to read .epub files
def read_epub_file(file_path):
    book = epub.read_epub(file_path)
    text = ""

    for doc in book.get_items_of_type(epub.ITEM_DOCUMENT):
        content = doc.get_content().decode('utf-8')
        soup = BeautifulSoup(content, 'html.parser')
        text += soup.get_text()

    return text

In [21]:
# Function to read .pdf files
def read_pdf_file(file_path):
    pdf = fitz.open(file_path)
    text = ""

    for page in pdf:
        text += page.get_text()

    return text

In [22]:
# Flag to indicate if the engine is currently speaking
is_speaking = False
engine_lock = threading.Lock()

# Playback Control Functions

def play_text():
    global current_text, is_speaking

    if not current_text.strip():
        messagebox.showwarning("No Text Loaded", "Please load a text file first.")
        return

    with engine_lock:
        if is_speaking:
            messagebox.showinfo("Already Playing", "The engine is already speaking.")
            return
        is_speaking = True

    try:
        threading.Thread(target=tts_engine, args=(current_text,), daemon=True).start()
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during playback:\n{e}")
        with engine_lock:
            is_speaking = False

def tts_engine(text):
    global is_speaking

    try:
        engine.say(text)
        engine.runAndWait()

    except RuntimeError as e:
        messagebox.showerror("Runtime Error", f"An error occurred: {e}")

    finally:
        with engine_lock:
            is_speaking = False

def stop_reading():
    global is_speaking
    
    with engine_lock:
        if is_speaking:
            engine.stop()
            is_speaking = False

In [23]:
# Speech Settings Functions 

def set_voice(event=None):
    voice_id = voice_combo.current()

    if voice_id >= 0 and voice_id < len(voices):
        engine.setProperty('voice', voices[voice_id].id)

def set_rate(event=None):
    try:
        rate = int(rate_slider.get())
        engine.setProperty('rate', rate)

    except ValueError:
        messagebox.showerror("Invalid Rate", "Please enter a valid integer for rate.")

def set_volume(event=None):
    try:
        volume = float(volume_slider.get())
        engine.setProperty('volume', volume)
        
    except ValueError:
        messagebox.showerror("Invalid Volume", "Please enter a valid float for volume.")

In [24]:
# Function to Save as Audio File
def save_to_audio():
    global current_text

    if not current_text:
        messagebox.showwarning("No Text Loaded", "Please load a text file first.")
        return
    
    file_path = filedialog.asksaveasfilename(defaultextension=".mp3",
                                             filetypes=[("MP3 Files", "*.mp3"), ("WAV Files", "*.wav")])
    if file_path:
        try:
            engine.save_to_file(current_text, file_path)
            engine.runAndWait()
            messagebox.showinfo("Success", f"Audio saved successfully at {file_path}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save audio:\n{e}")

In [25]:
# Function to Load File
def load_file():
    global current_text

    file_path = filedialog.askopenfilename(filetypes=[
        ("Supported Files", "*.txt *.TXT *.epub *.EPUB *.pdf *.PDF"),
        ("Text Files", "*.txt *.TXT"),
        ("EPUB Files", "*.epub *.EPUB"),
        ("PDF Files", "*.pdf *.PDF"),
        ("All Files", "*.*")
    ])

    if file_path:
        try:
            if file_path.endswith('.txt'):
                current_text = read_text_file(file_path)
            elif file_path.endswith('.epub'):
                current_text = read_epub_file(file_path)
            elif file_path.endswith('.pdf'):
                current_text = read_pdf_file(file_path)
            else:
                messagebox.showerror("Unsupported File", "Please select a .txt, .epub, or .pdf file.")
                return
            
            text_display.delete(1.0, tk.END)
            text_display.insert(tk.END, current_text)
            messagebox.showinfo("File Loaded", f"Successfully loaded {os.path.basename(file_path)}")

        except Exception as e:
            messagebox.showerror("Error", f"Failed to load file:\n{e}")

In [ ]:
# Initialize Tkinter
root = tk.Tk()
root.title("Text-to-Speech Application")
root.geometry("800x600")

# Create Frames for better layout management
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

middle_frame = tk.Frame(root)
middle_frame.pack(pady=10, fill="both", expand=True)

bottom_frame = tk.Frame(root)
bottom_frame.pack(pady=10)

# Load File Button
load_button = tk.Button(top_frame, text="Load File", command=load_file, width=20)
load_button.pack()

# Text Display Area with Scrollbar
text_display = tk.Text(middle_frame, wrap=tk.WORD, width=80, height=20)
text_display.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(middle_frame, command=text_display.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

text_display.config(yscrollcommand=scrollbar.set)

# Playback Control Buttons
play_button = tk.Button(bottom_frame, text="Play", command=play_text, width=10)
play_button.grid(row=0, column=0, padx=5, pady=5)

# Removed Pause and Resume Buttons as pyttsx3 does not support them
stop_button = tk.Button(bottom_frame, text="Stop", command=stop_reading, width=10)
stop_button.grid(row=0, column=1, padx=5, pady=5)

save_button = tk.Button(bottom_frame, text="Save as Audio", command=save_to_audio, width=15)
save_button.grid(row=0, column=2, padx=5, pady=5)

# Speech Settings
settings_frame = tk.LabelFrame(root, text="Speech Settings")
settings_frame.pack(pady=10, padx=10, fill="x")

# Voice Selection
tk.Label(settings_frame, text="Voice:").grid(row=0, column=0, padx=5, pady=5, sticky='e')
voice_combo = ttk.Combobox(settings_frame, state="readonly")
voice_names = [f"{voice.name} ({', '.join([lang.decode('utf-8') if isinstance(lang, bytes) else lang for lang in voice.languages])})" for voice in voices]
voice_combo['values'] = voice_names
voice_combo.current(0)  # Set default voice
voice_combo.grid(row=0, column=1, padx=5, pady=5, sticky='w')
voice_combo.bind("<<ComboboxSelected>>", set_voice)

# Speech Rate
tk.Label(settings_frame, text="Rate:").grid(row=1, column=0, padx=5, pady=5, sticky='e')
rate_slider = tk.Scale(settings_frame, from_=100, to=200, orient=tk.HORIZONTAL, command=lambda val: set_rate())
rate_slider.set(engine.getProperty('rate'))
rate_slider.grid(row=1, column=1, padx=5, pady=5, sticky='w')

# Volume Control
tk.Label(settings_frame, text="Volume:").grid(row=2, column=0, padx=5, pady=5, sticky='e')
volume_slider = tk.Scale(settings_frame, from_=0.0, to=1.0, resolution=0.1, orient=tk.HORIZONTAL, command=lambda val: set_volume())
volume_slider.set(engine.getProperty('volume'))
volume_slider.grid(row=2, column=1, padx=5, pady=5, sticky='w')

# Start the Tkinter event loop
root.mainloop()